In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

import optuna

%matplotlib inline

from colorama import Fore, Back, Style
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.3f}'.format)

import warnings
warnings.filterwarnings("ignore")

# Данные

In [2]:
data = pd.read_csv('data/_train_sem09 (1).csv')

In [3]:
data.head(2)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000,0.497,0.100,0.000,0.133,0.678,0.273,0.585,0.744,...,0,0,0,0,0,0,0,0,0,0
1,1,0.367,0.606,0.050,0.000,0.111,0.803,0.106,0.412,0.837,...,1,1,1,1,0,1,0,0,1,0


In [4]:
data.shape

(3751, 1777)

In [5]:
# Подготовка данных для следующей работы
X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

In [6]:
# В данную переменную буду сохранять результаты моделей [f1_train, f1_test, название]
f1_score_results = []
# Константы часто используемых параметров
RANDOM_STATE = 42
MAX_ITER = 500

In [7]:

def printAndSaveModelResults(model, methodName):
  """функция для вывода результатов score для train и test 
     и также для сохранения этих значений для последующего использования
  Args:
      model (model): Ссылка на обучепнную модель
      methodName (string): Название примененной модели
  """
  y_train_pred = model.predict(X_train)
  metrics_train = metrics.f1_score(y_train, y_train_pred).round(6)
  print(f"{Fore.LIGHTBLUE_EX}f1_score на тренировочном наборе: {Fore.LIGHTGREEN_EX}{metrics_train:.3f}{Fore.RESET}")
  y_test_pred = model.predict(X_test)
  metrics_test = metrics.f1_score(y_test, y_test_pred).round(6)
  print(f"{Fore.LIGHTBLUE_EX}f1_score на тестовом наборе: {Fore.LIGHTGREEN_EX}{metrics_test:.3f}{Fore.RESET}")

  f1_score_results.append({'result_train': metrics_train,
                           'result_test': metrics_test,
                          'method': methodName})

# Моделирование

Зафиксируем только метрики, которые были получены без дополнительной настройки, т.е со значениями гиперпараметров, установленных по умолчанию:

## Логистическая регрессия

In [7]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = MAX_ITER)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

printAndSaveModelResults(log_reg,'LogisticRegression native')



f1_score на тренировочном наборе: 0.885
f1_score на тестовом наборе: 0.781


## Случайный лес

In [8]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=RANDOM_STATE)

#Обучаем модель
rf.fit(X_train, y_train)

printAndSaveModelResults(rf,'RandomForestClassifier native')

f1_score на тренировочном наборе: 1.000
f1_score на тестовом наборе: 0.813


# Оптимизация гиперпараметров модели

### **GridSearchCV**

#### Логистическая регрессия

In [10]:
param_grid = {'penalty': ['l2', None] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 10, 10, dtype=float))},

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=RANDOM_STATE, max_iter=MAX_ITER), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  

%time grid_search.fit(X_train, y_train) 

printAndSaveModelResults(grid_search,'LogisticRegression GridSearch Optimization')

print(f"{Fore.LIGHTBLUE_EX} Наилучшие значения гиперпараметров: {Fore.LIGHTGREEN_EX}{grid_search.best_params_}{Fore.RESET}")


CPU times: total: 2.95 s
Wall time: 1min 8s
f1_score на тренировочном наборе: 0.522
f1_score на тестовом наборе: 0.545
 Наилучшие значения гиперпараметров: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}


#### Случайный лес

In [44]:
param_grid = {'n_estimators': list(range(100, 500, 50)),
              'min_samples_leaf': [3,5,7],
              'max_depth': list(np.linspace(2, 40, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=RANDOM_STATE), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_train, y_train) 

printAndSaveModelResults(grid_search_forest,'RandomForestClassifier GridSearch Optimization')

print(f"{Fore.LIGHTBLUE_EX}Наилучшие значения гиперпараметров: {Fore.LIGHTGREEN_EX}{grid_search_forest.best_params_}{Fore.RESET}")



CPU times: total: 5.12 s
Wall time: 36.1 s
f1_score на тренировочном наборе: 0.985
f1_score на тестовом наборе: 0.769
Наилучшие значения гиперпараметров: {'max_depth': 11, 'min_samples_leaf': 3, 'n_estimators': 350}


### **RandomizedSearchCV**

#### Логистическая регрессия

In [12]:
param_distributions = {'penalty': ['l2', None] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=RANDOM_STATE, max_iter=MAX_ITER), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 

printAndSaveModelResults(random_search,'LogisticRegression RandomSearch Optimization')

print(f"{Fore.LIGHTBLUE_EX} Наилучшие значения гиперпараметров: {Fore.LIGHTGREEN_EX}{random_search.best_params_}{Fore.RESET}")



CPU times: total: 4.19 s
Wall time: 24.6 s
f1_score на тренировочном наборе: 1.000
f1_score на тестовом наборе: 0.875
 Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.23}


#### Случайный лес

In [13]:
param_distributions = {'n_estimators': list(range(100, 500, 50)),
              'min_samples_leaf': [3,5,7],
              'max_depth': list(np.linspace(2, 40, 5, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator = ensemble.RandomForestClassifier(random_state=RANDOM_STATE), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 

printAndSaveModelResults(random_search_forest,'RandomForestClassifier RandomSearch Optimization')

print(f"{Fore.LIGHTBLUE_EX} Наилучшие значения гиперпараметров: {Fore.LIGHTGREEN_EX}{random_search_forest.best_params_}{Fore.RESET}")



CPU times: total: 2.78 s
Wall time: 15.2 s
f1_score на тренировочном наборе: 0.985
f1_score на тестовом наборе: 0.769
 Наилучшие значения гиперпараметров: {'n_estimators': 350, 'min_samples_leaf': 3, 'max_depth': 40}


### **Hyperopt**

##### Подготовка функций для расчета

In [14]:
# Как то hyperopt не зажелал норм работать с hp.не помню как, поэтому триканул с hp.choice
# обьявил отдельно, так как в предсказании, метод выдает индексы категорий, а не сами категории,
# поэтому обьявил параметры отдельными переменными
penalty = ['l2', None]
solver = ['lbfgs', 'sag']
C = np.arange(.1,1.1,.1)

space = {'penalty': hp.choice('penalty', penalty) ,
          'solver': hp.choice('solver', solver),
          'C': hp.choice('C', C)
          }

scoreCalc = 'standard'
modelType = 'logReg'

In [15]:
def hyperparameter_tuning(params):
    """Функция для проведения оптимизации библиотекой Hyperopt
       так же внутри проверяем для какого типа модели провести гиперпараметризацию 
       и с кроссвалидацией или без 
    Args:
        params (params): Обьект с параметрами для обучения

    Returns:
        object: Обьект с результатом 
    """
    if modelType =='logReg':
        model = linear_model.LogisticRegression(**params, random_state=RANDOM_STATE, max_iter=MAX_ITER,n_jobs=-1)
    elif modelType == 'ranFor':
        model = ensemble.RandomForestClassifier(**params, random_state=RANDOM_STATE, n_jobs=-1)
    if scoreCalc == 'standard':
        model.fit(X_train, y_train)
        score = metrics.f1_score(y_test, model.predict(X_test))
    elif scoreCalc == 'cross':
        score = cross_val_score(model, X_train, y_train, cv = 5, scoring="f1", n_jobs = -1).mean()
    return {"loss": -score, "status": STATUS_OK}

#### Логистическая регрессия

##### Стандартный расчет score

In [ ]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=10, 
    trials=trials
)


In [ ]:
log_reg_Hyperopt = linear_model.LogisticRegression(
    random_state=RANDOM_STATE, 
    penalty=penalty[int(best['penalty'])],
    solver=solver[int(best['solver'])],
    C=C[int(best['C'])]
)

#Обучаем модель
log_reg_Hyperopt.fit(X_train, y_train)

printAndSaveModelResults(log_reg_Hyperopt,'LogisticRegression Hyperopt Optimization')

f1_score на тренировочном наборе: 1.000
f1_score на тестовом наборе: 0.933


##### Кросс-валидация

In [ ]:
# Initialize trials object
scoreCalc = 'cross'

trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=10, 
    trials=trials
)


100%|██████████| 10/10 [02:04<00:00, 12.45s/trial, best loss: -0.7050727050727051]


In [ ]:
log_reg_Hyperopt_cross = linear_model.LogisticRegression(
    random_state=RANDOM_STATE, 
    penalty=penalty[int(best['penalty'])],
    solver=solver[int(best['solver'])],
    C=C[int(best['C'])]
)

#Обучаем модель
log_reg_Hyperopt_cross.fit(X_train, y_train)

printAndSaveModelResults(log_reg_Hyperopt_cross,'LogisticRegression Hyperopt Optimization with Cross Validation')

f1_score на тренировочном наборе: 1.000
f1_score на тестовом наборе: 0.875


#### Случайный лес

In [ ]:
n_estimators = np.arange(100, 500, 2, dtype = int)
max_depth = np.arange(15, 26, 1, dtype = int)
min_samples_leaf = np.arange(2, 11, 1, dtype=int)

# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.choice('n_estimators', n_estimators ),
       'max_depth' : hp.choice('max_depth', max_depth),
       'min_samples_leaf': hp.choice('min_samples_leaf', min_samples_leaf)
      }
scoreCalc = 'standard'
modelType = 'ranFor'

##### Стандартный расчет score

In [ ]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=10, 
    trials=trials
)

100%|██████████| 10/10 [00:06<00:00,  1.55trial/s, best loss: -0.8571428571428571]


In [ ]:
rf_Hyperopt = ensemble.RandomForestClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=n_estimators[int(best['n_estimators'])],
    max_depth=max_depth[int(best['max_depth'])],
    min_samples_leaf=min_samples_leaf[int(best['min_samples_leaf'])],
)

#Обучаем модель
rf_Hyperopt.fit(X_train, y_train)

printAndSaveModelResults(rf_Hyperopt,'RandomForestClassifier Hyperopt Optimization')

f1_score на тренировочном наборе: 1.000
f1_score на тестовом наборе: 0.857


##### Кросс-валидация

In [ ]:
scoreCalc = 'cross'
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=10, 
    trials=trials
)

100%|██████████| 10/10 [00:14<00:00,  1.49s/trial, best loss: -0.48666666666666664]


In [ ]:
rf_Hyperopt_cross = ensemble.RandomForestClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=n_estimators[int(best['n_estimators'])],
    max_depth=max_depth[int(best['max_depth'])],
    min_samples_leaf=min_samples_leaf[int(best['min_samples_leaf'])],
)

#Обучаем модель
rf_Hyperopt_cross.fit(X_train, y_train)

printAndSaveModelResults(rf_Hyperopt_cross,'RandomForestClassifier Hyperopt Optimization with Crossvalidation')

f1_score на тренировочном наборе: 0.985
f1_score на тестовом наборе: 0.769


## Optuna

##### Подготовка функций для расчета

In [33]:
scoreCalc = 'standard'
modelType = 'logReg'

In [26]:
def optuna_optimize(trial):
  
  # задаем пространства поиска гиперпараметров
  if modelType == 'logReg':
    penalty = ['l2', 'none']
    solver = ['lbfgs', 'sag']
    C = np.arange(.1,1.1,.1)
  
    penalty=trial.suggest_categorical('penalty', penalty)
    solver=trial.suggest_categorical('solver',solver)
    C=trial.suggest_categorical('C',C)
    
    model = linear_model.LogisticRegression(
      penalty = penalty,
      solver = solver,
      C = C,
      random_state = RANDOM_STATE,
      n_jobs=-1
    )

  elif modelType == 'ranFor':
    n_estimators = trial.suggest_int('n_estimators', 100, 310, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)
    max_depth = trial.suggest_int('max_depth', 15, 40, 1)

    # создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=RANDOM_STATE,
                                          n_jobs=-1
                                          )
  # обучаем модель
  if scoreCalc == 'standard':
    model.fit(X_train, y_train)
    score = metrics.f1_score(y_test, model.predict(X_test))
  elif scoreCalc == 'cross':
      score = cross_val_score(model, X_train, y_train, cv = 5, scoring="f1", n_jobs = -1).mean()  
  return score

#### Логистическая регрессия

##### Стандартный расчет score

In [ ]:
%time

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_logreg = optuna.create_study(study_name="LogisticRegression Optuna Optimization", 
                                   direction="maximize",
                                   storage='sqlite:///ex.db',
                                   load_if_exists=True
                                   )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_logreg.optimize(optuna_optimize, n_trials=1000
                      , n_jobs=-1
                      )

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_logreg.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_logreg.best_value))

In [13]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study_logreg.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'LogisticRegression Optuna Optimization')

f1_score на тренировочном наборе: 0.863
f1_score на тестовом наборе: 0.783


##### Кросс-валидация

In [ ]:
%time

scoreCalc = "cross"
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_logreg_cross = optuna.create_study(study_name="LogisticRegression Optuna Optimization with Cross Validation",
                                         direction="maximize",
                                         storage='sqlite:///ex.db',
                                         load_if_exists=True,
                                         )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_logreg_cross.optimize(optuna_optimize, n_trials=20, n_jobs=-1)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_logreg_cross.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_logreg_cross.best_value))

In [17]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study_logreg_cross.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'LogisticRegression Optuna Optimization with Cross Validation')

f1_score на тренировочном наборе: 0.862
f1_score на тестовом наборе: 0.781


#### Случайный лес

##### Стандартный расчет score

In [32]:
%time

scoreCalc = "standard"
modelType = "ranFor"
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_ranFor = optuna.create_study(study_name="RandomForestClassifier Optuna Optimization",
                                   direction="maximize",
                                  #  storage='sqlite:///ex.db',
                                   load_if_exists=False,
                                   )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_ranFor.optimize(optuna_optimize, n_trials=1000)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_ranFor.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_ranFor.best_value))

[I 2023-04-28 11:41:03,635] A new study created in memory with name: RandomForestClassifier Optuna Optimization


CPU times: total: 0 ns
Wall time: 0 ns


[I 2023-04-28 11:41:04,199] Trial 0 finished with value: 0.828978622327791 and parameters: {'n_estimators': 180, 'min_samples_leaf': 3, 'max_depth': 17}. Best is trial 0 with value: 0.828978622327791.
[I 2023-04-28 11:41:04,822] Trial 1 finished with value: 0.8246445497630331 and parameters: {'n_estimators': 220, 'min_samples_leaf': 4, 'max_depth': 26}. Best is trial 0 with value: 0.828978622327791.
[I 2023-04-28 11:41:05,232] Trial 2 finished with value: 0.8293838862559241 and parameters: {'n_estimators': 130, 'min_samples_leaf': 5, 'max_depth': 40}. Best is trial 2 with value: 0.8293838862559241.
[I 2023-04-28 11:41:05,870] Trial 3 finished with value: 0.8246445497630331 and parameters: {'n_estimators': 230, 'min_samples_leaf': 4, 'max_depth': 34}. Best is trial 2 with value: 0.8293838862559241.
[I 2023-04-28 11:41:06,339] Trial 4 finished with value: 0.8182912154031289 and parameters: {'n_estimators': 150, 'min_samples_leaf': 6, 'max_depth': 30}. Best is trial 2 with value: 0.829383

Наилучшие значения гиперпараметров {'n_estimators': 100, 'min_samples_leaf': 5, 'max_depth': 23}
f1_score на обучающем наборе: 0.83


In [33]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_ranFor.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'RandomForestClassifier Optuna Optimization')

f1_score на тренировочном наборе: 0.943
f1_score на тестовом наборе: 0.835


##### Кросс-валидация

In [37]:
%time

scoreCalc = "cross"

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_ranFor_cross = optuna.create_study(study_name="RandomForestClassifier Optuna Optimization with Cross Validation",
                                   direction="maximize",
                              #      storage='sqlite:///ex.db',
                              #      load_if_exists=True,
                                         )
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_ranFor_cross.optimize(optuna_optimize, n_trials=1000, n_jobs=-1)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_ranFor_cross.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_ranFor_cross.best_value))

[I 2023-04-28 12:59:35,503] A new study created in memory with name: RandomForestClassifier Optuna Optimization with Cross Validation


CPU times: total: 0 ns
Wall time: 0 ns


[I 2023-04-28 12:59:50,145] Trial 6 finished with value: 0.7991909791462259 and parameters: {'n_estimators': 190, 'min_samples_leaf': 7, 'max_depth': 27}. Best is trial 6 with value: 0.7991909791462259.
[I 2023-04-28 12:59:51,506] Trial 7 finished with value: 0.7991485335851993 and parameters: {'n_estimators': 120, 'min_samples_leaf': 6, 'max_depth': 33}. Best is trial 6 with value: 0.7991909791462259.
[I 2023-04-28 12:59:56,881] Trial 8 finished with value: 0.8053125585369955 and parameters: {'n_estimators': 140, 'min_samples_leaf': 4, 'max_depth': 37}. Best is trial 8 with value: 0.8053125585369955.
[I 2023-04-28 12:59:57,668] Trial 41 finished with value: 0.7972448111967132 and parameters: {'n_estimators': 100, 'min_samples_leaf': 7, 'max_depth': 39}. Best is trial 8 with value: 0.8053125585369955.
[I 2023-04-28 12:59:58,372] Trial 1 finished with value: 0.8022259041556117 and parameters: {'n_estimators': 100, 'min_samples_leaf': 4, 'max_depth': 19}. Best is trial 8 with value: 0.80

Наилучшие значения гиперпараметров {'n_estimators': 280, 'min_samples_leaf': 3, 'max_depth': 31}
f1_score на обучающем наборе: 0.81


In [39]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_ranFor_cross.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

printAndSaveModelResults(model,'RandomForestClassifier Optuna Optimization with Cross Validation')

f1_score на тренировочном наборе: 0.978
f1_score на тестовом наборе: 0.823


In [24]:
optuna.visualization.is_available()

True

Немного графиков для одной из выборок Optuna. 

In [25]:
optuna.visualization.plot_optimization_history(study_ranFor, target_name="f1_score")

In [26]:
optuna.visualization.plot_param_importances(study_ranFor, target_name="f1_score")

In [27]:
optuna.visualization.plot_contour(study_ranFor, params=["max_depth", "min_samples_leaf"],
                                  target_name="f1_score")

In [28]:
resultf1ScoreData = pd.DataFrame(f1_score_results).set_index('method').sort_values(by='result_test',ascending=False)

display(resultf1ScoreData.\
        style.background_gradient(\
        axis=0, cmap='GnBu',
        low = 0.8,
        high=0.9,
        vmin = 0.8,
        vmax = 1))

KeyError: '`Styler.apply` and `.applymap` are not compatible with non-unique index or columns.'

## Немного выводов (хотя по заданию их вроде не надо)

* Обе модели(Логистической регрессии и Случайный лес) показали не самые худшие результаты. Т.е значения обучения по умолчанию, можно сказать, подобраны достаточно оптимально
* Также, важно отметить, что к подбору параметров также надо относится "с головой", так как видно из таблицы выше, мы также и испортили предсказательные особенности в некоторых случаях
* В нашем случае LogisticRegression в большинстве случаев показала лучшие результаты

Попробуем немного поиграться на отсающем RandomForestClassifier

Учитывая что f1_train всегда очень высокая, поменяю расчет f1_score на test.
Также, увеличу диапазон минимального количества листьев, так как алгоритм считает это важным параметром.

In [ ]:
def optuna_optimize_improvement(trial):
  
  n_estimators = trial.suggest_int('n_estimators', 100, 310, 10)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 15, 1)
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)

    # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=RANDOM_STATE)
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_test, model.predict(X_test))
  # score = cross_val_score(model, X_train, y_train, cv = 5, scoring="f1", n_jobs = -1).mean()  
  return score

In [ ]:
%time

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_ranFor_opt = optuna.create_study(study_name="RandomForestClassifier Optuna Optimization Improved", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_ranFor_opt.optimize(optuna_optimize_improvement, n_trials=50)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_ranFor_opt.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_ranFor_opt.best_value))

[I 2023-04-20 20:15:51,904] A new study created in memory with name: RandomForestClassifier Optuna Optimization Improved


CPU times: total: 0 ns
Wall time: 0 ns


[I 2023-04-20 20:15:53,729] Trial 0 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 310, 'min_samples_leaf': 7, 'max_depth': 22}. Best is trial 0 with value: 0.6666666666666666.
[I 2023-04-20 20:15:55,660] Trial 1 finished with value: 0.8571428571428571 and parameters: {'n_estimators': 300, 'min_samples_leaf': 1, 'max_depth': 39}. Best is trial 1 with value: 0.8571428571428571.
[I 2023-04-20 20:15:56,902] Trial 2 finished with value: 0.8571428571428571 and parameters: {'n_estimators': 210, 'min_samples_leaf': 2, 'max_depth': 19}. Best is trial 1 with value: 0.8571428571428571.
[I 2023-04-20 20:15:58,443] Trial 3 finished with value: 0.0 and parameters: {'n_estimators': 280, 'min_samples_leaf': 14, 'max_depth': 27}. Best is trial 1 with value: 0.8571428571428571.
[I 2023-04-20 20:15:59,437] Trial 4 finished with value: 0.9333333333333333 and parameters: {'n_estimators': 160, 'min_samples_leaf': 1, 'max_depth': 21}. Best is trial 4 with value: 0.9333333333333333.

Наилучшие значения гиперпараметров {'n_estimators': 160, 'min_samples_leaf': 1, 'max_depth': 21}
f1_score на обучающем наборе: 0.93


Как мы видим один из лучших вариантов имеет минимальное кол-во листьев 1. До этого мы рассматривали только диапазон [3,5,7]

In [ ]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_ranFor_opt.best_params,random_state=RANDOM_STATE, )
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=21, n_estimators=160, random_state=42)

In [ ]:
y_train_pred = model.predict(X_train)
metrics_train = metrics.f1_score(y_train, y_train_pred).round(6)
print(f"{Fore.LIGHTBLUE_EX}f1_score на тренировочном наборе: {Fore.LIGHTGREEN_EX}{metrics_train:.3f}{Fore.RESET}")
y_test_pred = model.predict(X_test)
metrics_test = metrics.f1_score(y_test, y_test_pred).round(6)
print(f"{Fore.LIGHTBLUE_EX}f1_score на тестовом наборе: {Fore.LIGHTGREEN_EX}{metrics_test:.3f}{Fore.RESET}")


f1_score на тренировочном наборе: 1.000
f1_score на тестовом наборе: 0.933


In [ ]:
f1_score_results.append({'result_train': metrics_train,
                           'result_test': metrics_test,
                          'method': 'RandomForestClassifier Optuna Optimization Improved'})

Посмотрим еще раз на результаты
Как видно, путем "разумной" оптимизации гиперпараметров, мы получили ТОП 1 результат по f1_score метрике для RandomForestClassifier

In [ ]:
resultf1ScoreData = pd.DataFrame(f1_score_results).set_index('method').sort_values(by='result_test',ascending=False)

display(resultf1ScoreData.\
        style.background_gradient(\
        axis=0, cmap='GnBu',
        low = 0.8,
        high=0.9,
        vmin = 0.8,
        vmax = 1))

,result_train,result_test
method,,
LogisticRegression Hyperopt Optimization,1.000000,0.933333
RandomForestClassifier Optuna Optimization Improved,1.000000,0.933333
LogisticRegression RandomSearch Optimization,1.000000,0.875000
LogisticRegression Hyperopt Optimization with Cross Validation,1.000000,0.875000
LogisticRegression Optuna Optimization with Cross Validation,1.000000,0.875000
RandomForestClassifier native,1.000000,0.857143
RandomForestClassifier Hyperopt Optimization,1.000000,0.857143
LogisticRegression native,1.000000,0.823529
LogisticRegression GridSearch Optimization,1.000000,0.823529


И напоследок графики

* Как видно, максимального f1_score мы уже добились примерно на 5-6 иттерации
* Важность минимального кол-ва листьев выросла аж до 99%
* Четко проглядывается, что f1_score улучшается при уменьшении листьев до 1-2 и увеличением глубины дерева. Ярко выраженные две области в правом нижнем углу на последнем графике

Так что можно смело заявить, оптимизация гиперпараметров может достаточно сильно улучшить качество модели, при правильном использовании

In [ ]:
optuna.visualization.plot_optimization_history(study_ranFor_opt, target_name="f1_score")

In [ ]:
optuna.visualization.plot_param_importances(study_ranFor_opt, target_name="f1_score")

In [ ]:
optuna.visualization.plot_contour(study_ranFor_opt, params=["max_depth", "min_samples_leaf"],
                                  target_name="f1_score")